In [1]:
import PyPDF2
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [2]:
# Open the PDF file
pdf_file = open('hh.pdf', 'rb')

In [15]:
# Read the PDF file and extract the text
pdf_reader = PyPDF2.PdfReader(pdf_file)
text = ''
for page in range(len(pdf_reader.pages)):
    text += pdf_reader.pages[page].extract_text()

# Load the question-answering model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = AutoModelForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Define the questions to answer
questions = [
    {
        'question': 'What is the name of applicant?',
        'context': text
    },
    {
        'question': 'what is applicant birth date?',
        'context': text
    },
    {    
        'question': 'what is the application date?',
        'context': text
    },
    {    
        'question': 'what is the passport number?',
        'context': text
    }
]

# Process each question and extract the answers
for question in questions:
    # Tokenize the input text
    inputs = tokenizer(question['question'], question['context'], return_tensors='pt',max_length=512,truncation=True)
    
    # Get the start and end logits for the answer
    with torch.no_grad():
        start_logits, end_logits = model(**inputs).values()

    # Find the start and end positions with the highest logits
    start_index = torch.argmax(start_logits, dim=1).item()
    end_index = torch.argmax(end_logits, dim=1).item()
    
    # Extract the answer from the input text
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_index:end_index+1]))
    
    # Print the question and answer
    print(f"Q: {question['question']}")
    print(f"A: {answer}")

Q: What is the name of applicant?
A: hatem samir issa haddad
Q: what is applicant birth date?
A: 3 august 1970
Q: what is the application date?
A: 1 july 2022
Q: what is the passport number?
A: p669481


In [ ]:
import PyPDF2
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the question-answering model and tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = AutoModelForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# Open the PDF file and extract the text
with open('hh.pdf', 'rb') as f:
    pdf_reader = PyPDF2.PdfReader(f)
    text = ''
    for page in range(len(pdf_reader.pages)):
        text += pdf_reader.pages[page].extract_text()

# Define a function to answer questions
def answer_question(question):
    # Tokenize the input text
    inputs = tokenizer(question, text, return_tensors='pt', max_length=512, truncation=True)
    
    # Get the start and end logits for the answer
    with torch.no_grad():
        start_logits, end_logits = model(**inputs).values()

    # Find the start and end positions with the highest logits
    start_index = torch.argmax(start_logits, dim=1).item()
    end_index = torch.argmax(end_logits, dim=1).item()
    
    # Extract the answer from the input text
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0][start_index:end_index+1]))
    
    return answer

# Define a function to run the chatbot
def run_chatbot():
    print('Chatbot: Hi, I can answer questions about this PDF file. What would you like to know?')
    while True:
        user_input = input('User: ')
        if user_input.lower() == 'quit':
            print('Chatbot: Goodbye!')
            break
        answer = answer_question(user_input)
        print(f'Chatbot: {answer}')

# Run the chatbot
run_chatbot()


Chatbot: Hi, I can answer questions about this PDF file. What would you like to know?
User: what is the applicant name
Chatbot: hatem samir issa haddad page 1 of 7applicant ' s full name : hatem samir issa haddad
User: when is his birth date
Chatbot: 3 august 1970
User: what is his passport number
Chatbot: p669481
User: what is his email
Chatbot: arabicdatamining @ gmail. com
User: fees
Chatbot: [SEP]
User: what is the fees
Chatbot: 480. 00 usd
User: what is the visa category
Chatbot: visitor
User: how long is the visa
Chatbot: 2 years
